# Import in data:


In [0]:
import json
import requests

import pandas as pd

API_KEY = ""  # Put strainapi key here!

In [0]:
query = "http://strainapi.evanbusse.com/" + API_KEY + "/strains/search/all"
response = requests.get(query)

In [3]:
print("Status code:", response.status_code)
# print("Content:", response.content)

Status code: 200


In [4]:
data = json.loads(response.content)

# print(data)
print(type(data))

<class 'dict'>


# Examine data:

In [5]:
effects_list = []
medical_list = []
negative_list = []
flavor_list = []

for item in data.keys():
    for effect in data[item]['effects']['positive']:
        if effect not in effects_list: 
            effects_list.append(effect)

    for medical in data[item]['effects']['medical']:
        if medical not in medical_list:
            medical_list.append(medical)

    for negative in data[item]['effects']['negative']:
        if negative not in negative_list:
            negative_list.append(negative)
    
    for flavor in data[item]['flavors']:
        if flavor not in flavor_list:
            flavor_list.append(flavor)

print("Number of positive effects:", len(effects_list))
for item in effects_list:
    print(item)

print()

print("Number of negative effects:", len(negative_list))
for item in negative_list:
    print(item)

print()

print("Number of medical symptoms:", len(medical_list))
for item in medical_list:
    print(item)

print()

print("Number of flavors:", len(flavor_list))
for item in flavor_list:
    print(item)

Number of positive effects: 13
Relaxed
Hungry
Happy
Sleepy
Euphoric
Creative
Energetic
Talkative
Uplifted
Tingly
Focused
Giggly
Aroused

Number of negative effects: 5
Dizzy
Dry Mouth
Paranoid
Dry Eyes
Anxious

Number of medical symptoms: 15
Depression
Insomnia
Pain
Stress
Lack of Appetite
Nausea
Headache
Cramps
Fatigue
Headaches
Muscle Spasms
Eye Pressure
Inflammation
Spasticity
Seizures

Number of flavors: 48
Earthy
Chemical
Pine
Spicy/Herbal
Pungent
Pepper
Flowery
Citrus
Orange
Sweet
Skunk
Woody
Grape
Minty
Cheese
Diesel
Tropical
Grapefruit
Nutty
Lemon
Berry
Blueberry
Ammonia
Apple
Rose
Butter
Mango
Honey
Tea
Lime
Lavender
Strawberry
Mint
Chestnut
Tree Fruit
Pear
Apricot
Peach
Blue Cheese
Menthol
Coffee
Tar
Pineapple
Sage
Vanilla
Plum
Tobacco
Violet


In [0]:
df = pd.DataFrame.from_dict(data, orient='index')

df = df.reset_index()
df = df.set_index('id')
df = df.rename(columns={'index': 'name'})

In [7]:
df.head()

,name,race,flavors,effects
id,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


# Expand effects column:

In [8]:
df = pd.concat([df.drop(['effects'], axis=1), df['effects'].apply(pd.Series)], axis=1)
df.head()

,name,race,flavors,positive,negative,medical
id,,,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A..."
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N..."
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H..."
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M..."
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H..."


In [9]:
df.isnull().sum().sum()

0

# Import in rating and description data:

In [10]:
# Obtained from: https://www.kaggle.com/kingburrito666/cannabis-strains#cannabis.csv

new_data = pd.read_csv('cannabis.csv')

new_data['Strain'] = new_data['Strain'].str.replace("-", " ")
new_data = new_data.rename(columns={"Strain": "name"})
new_data = new_data.drop(columns=['Effects', 'Flavor', 'Type'])
new_data = new_data.rename(columns={'Description': 'description'})
new_data = new_data.rename(columns={'Rating': 'rating'})
new_data['description'] = new_data['description'].fillna("")

new_data.head()

,name,rating,description
0,100 Og,4.0,$100 OG is a 50/50 hybrid strain that packs a ...
1,98 White Widow,4.7,The ‘98 Aloha White Widow is an especially pot...
2,1024,4.4,1024 is a sativa-dominant hybrid bred in Spain...
3,13 Dawgs,4.2,13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K Gold,4.6,"Also known as Kosher Tangie, 24k Gold is a 60%..."


In [11]:
df = df.join(new_data.set_index('name'), on='name', how='inner')

print("Shape after inner merge:", df.shape)
df.head()

Shape after inner merge: (1495, 8)


,name,race,flavors,positive,negative,medical,rating,description
id,,,,,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",4.2,"Afpak, named for its direct Afghani and Pakist..."
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",3.9,African refers to the indigenous varieties of ...
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",4.8,"Afternoon Delight, created by Colorado Seed In..."
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",4.2,Afwreck is a hybrid cross of Afghani and Train...
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",4.2,Don’t let the name scare you! The only herbici...


# Create boolean columns for easier searching:

In [0]:
def list_into_columns(data, column_name):
    new_data = pd.DataFrame(data[column_name])
    new_data[column_name] = new_data[column_name].apply(', '.join)
    new_data = pd.concat([new_data,new_data[column_name].str.get_dummies(sep=', ')],1)
    new_data = new_data.drop([column_name], axis=1)
    new_data = new_data.astype('bool')
    data = df.join(new_data, how='left')
    # data = data.drop([column_name], axis=1)
    
    return data

In [13]:
df = list_into_columns(df, 'positive')

df = df.rename(columns={
    "Aroused": "positive_aroused",
    "Creative": "positive_creative",
    "Energetic": "positive_energetic",
    "Euphoric": "positive_euphoric",
    "Focused": "positive_focused",
    "Giggly": "positive_giggly",
    "Happy": "positive_happy",
    "Hungry": "positive_hungry",
    "Relaxed": "positive_relaxed",
    "Sleepy": "positive_sleepy",
    "Talkative": "positive_talkative",
    "Tingly": "positive_tingly",
    "Uplifted": "positive_uplifted",
    })

df.head()

,name,race,flavors,positive,negative,medical,rating,description,positive_aroused,positive_creative,positive_energetic,positive_euphoric,positive_focused,positive_giggly,positive_happy,positive_hungry,positive_relaxed,positive_sleepy,positive_talkative,positive_tingly,positive_uplifted
id,,,,,,,,,,,,,,,,,,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",4.2,"Afpak, named for its direct Afghani and Pakist...",False,False,False,False,False,False,True,True,True,True,False,False,False
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",3.9,African refers to the indigenous varieties of ...,False,True,True,True,False,False,True,False,False,False,True,False,False
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",4.8,"Afternoon Delight, created by Colorado Seed In...",False,False,False,True,False,False,False,True,True,False,False,True,True
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",4.2,Afwreck is a hybrid cross of Afghani and Train...,False,True,False,False,False,False,True,False,True,True,False,False,True
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",4.2,Don’t let the name scare you! The only herbici...,False,False,True,True,False,False,True,False,True,False,False,False,True


In [14]:
df = list_into_columns(df, 'negative')

df = df.rename(columns={
    "Anxious": "negative_anxious",
    "Dizzy": "negative_dizzy",
    "Dry Eyes": "negative_dry_eyes",
    "Dry Mouth": "negative_dry_mouth",
    "Paranoid": "negative_paranoid",
    })

df.head()

,name,race,flavors,positive,negative,medical,rating,description,positive_aroused,positive_creative,positive_energetic,positive_euphoric,positive_focused,positive_giggly,positive_happy,positive_hungry,positive_relaxed,positive_sleepy,positive_talkative,positive_tingly,positive_uplifted,negative_anxious,negative_dizzy,negative_dry_eyes,negative_dry_mouth,negative_paranoid
id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",4.2,"Afpak, named for its direct Afghani and Pakist...",False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,False,False,False
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",3.9,African refers to the indigenous varieties of ...,False,True,True,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",4.8,"Afternoon Delight, created by Colorado Seed In...",False,False,False,True,False,False,False,True,True,False,False,True,True,False,True,False,True,True
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",4.2,Afwreck is a hybrid cross of Afghani and Train...,False,True,False,False,False,False,True,False,True,True,False,False,True,False,True,True,True,True
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",4.2,Don’t let the name scare you! The only herbici...,False,False,True,True,False,False,True,False,True,False,False,False,True,False,True,True,True,True


In [15]:
df = list_into_columns(df, 'medical')

# Merge "headache" and "headaches" columns
df['Headaches'] = df['Headaches'] | df['Headache']
df = df.drop(columns=["Headache"])

df = df.rename(columns={
    "Cramps": "symptom_cramps",
    "Depression": "symptom_depression",
    "Eye Pressure": "symptom_eye_pressure",
    "Fatigue": "symptom_fatigue",
    "Headaches": "symptom_headaches",
    "Inflammation": "symptom_inflammation",
    "Insomnia": "symptom_insomnia",
    "Lack of Appetite": "symptom_lack_of_appetite",
    "Muscle Spasms": "symptom_muscule_spasms",
    "Nausea": "symptom_nausea",
    "Pain": "symptom_pain",
    "Seizures": "symptom_seizures",
    "Spasticity": "symptom_spasticity",
    "Stress": "symptom_stress",
    })

df.head()

,name,race,flavors,positive,negative,medical,rating,description,positive_aroused,positive_creative,positive_energetic,positive_euphoric,positive_focused,positive_giggly,positive_happy,positive_hungry,positive_relaxed,positive_sleepy,positive_talkative,positive_tingly,positive_uplifted,negative_anxious,negative_dizzy,negative_dry_eyes,negative_dry_mouth,negative_paranoid,symptom_cramps,symptom_depression,symptom_eye_pressure,symptom_fatigue,symptom_headaches,symptom_inflammation,symptom_insomnia,symptom_lack_of_appetite,symptom_muscule_spasms,symptom_nausea,symptom_pain,symptom_seizures,symptom_spasticity,symptom_stress
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Afpak,hybrid,"[Earthy, Chemical, Pine]","[Relaxed, Hungry, Happy, Sleepy]",[Dizzy],"[Depression, Insomnia, Pain, Stress, Lack of A...",4.2,"Afpak, named for its direct Afghani and Pakist...",False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,True
2,African,sativa,"[Spicy/Herbal, Pungent, Earthy]","[Euphoric, Happy, Creative, Energetic, Talkative]",[Dry Mouth],"[Depression, Pain, Stress, Lack of Appetite, N...",3.9,African refers to the indigenous varieties of ...,False,True,True,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,False,False,True
3,Afternoon Delight,hybrid,"[Pepper, Flowery, Pine]","[Relaxed, Hungry, Euphoric, Uplifted, Tingly]","[Dizzy, Dry Mouth, Paranoid]","[Depression, Insomnia, Pain, Stress, Cramps, H...",4.8,"Afternoon Delight, created by Colorado Seed In...",False,False,False,True,False,False,False,True,True,False,False,True,True,False,True,False,True,True,True,True,False,False,True,False,True,False,False,False,True,False,False,True
4,Afwreck,hybrid,"[Pine, Earthy, Flowery]","[Relaxed, Happy, Creative, Uplifted, Sleepy]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Pain, Stress, Headache, Fatigue, Headaches, M...",4.2,Afwreck is a hybrid cross of Afghani and Train...,False,True,False,False,False,False,True,False,True,True,False,False,True,False,True,True,True,True,False,False,False,True,True,False,False,False,True,False,True,False,False,True
5,Agent Orange,hybrid,"[Citrus, Orange, Sweet]","[Relaxed, Euphoric, Happy, Energetic, Uplifted]","[Dizzy, Dry Mouth, Paranoid, Dry Eyes]","[Depression, Pain, Stress, Nausea, Headache, H...",4.2,Don’t let the name scare you! The only herbici...,False,False,True,True,False,False,True,False,True,False,False,False,True,False,True,True,True,True,False,True,False,False,True,False,False,False,False,True,True,False,False,True


In [16]:
df['flavors'] = df['flavors'].apply(', '.join)
df['positive'] = df['positive'].apply(', '.join)
df['negative'] = df['negative'].apply(', '.join)
df['medical'] = df['medical'].apply(', '.join)

df.head()

,name,race,flavors,positive,negative,medical,rating,description,positive_aroused,positive_creative,positive_energetic,positive_euphoric,positive_focused,positive_giggly,positive_happy,positive_hungry,positive_relaxed,positive_sleepy,positive_talkative,positive_tingly,positive_uplifted,negative_anxious,negative_dizzy,negative_dry_eyes,negative_dry_mouth,negative_paranoid,symptom_cramps,symptom_depression,symptom_eye_pressure,symptom_fatigue,symptom_headaches,symptom_inflammation,symptom_insomnia,symptom_lack_of_appetite,symptom_muscule_spasms,symptom_nausea,symptom_pain,symptom_seizures,symptom_spasticity,symptom_stress
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Afpak,hybrid,"Earthy, Chemical, Pine","Relaxed, Hungry, Happy, Sleepy",Dizzy,"Depression, Insomnia, Pain, Stress, Lack of Ap...",4.2,"Afpak, named for its direct Afghani and Pakist...",False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,True
2,African,sativa,"Spicy/Herbal, Pungent, Earthy","Euphoric, Happy, Creative, Energetic, Talkative",Dry Mouth,"Depression, Pain, Stress, Lack of Appetite, Na...",3.9,African refers to the indigenous varieties of ...,False,True,True,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,False,False,True
3,Afternoon Delight,hybrid,"Pepper, Flowery, Pine","Relaxed, Hungry, Euphoric, Uplifted, Tingly","Dizzy, Dry Mouth, Paranoid","Depression, Insomnia, Pain, Stress, Cramps, He...",4.8,"Afternoon Delight, created by Colorado Seed In...",False,False,False,True,False,False,False,True,True,False,False,True,True,False,True,False,True,True,True,True,False,False,True,False,True,False,False,False,True,False,False,True
4,Afwreck,hybrid,"Pine, Earthy, Flowery","Relaxed, Happy, Creative, Uplifted, Sleepy","Dizzy, Dry Mouth, Paranoid, Dry Eyes","Pain, Stress, Headache, Fatigue, Headaches, Mu...",4.2,Afwreck is a hybrid cross of Afghani and Train...,False,True,False,False,False,False,True,False,True,True,False,False,True,False,True,True,True,True,False,False,False,True,True,False,False,False,True,False,True,False,False,True
5,Agent Orange,hybrid,"Citrus, Orange, Sweet","Relaxed, Euphoric, Happy, Energetic, Uplifted","Dizzy, Dry Mouth, Paranoid, Dry Eyes","Depression, Pain, Stress, Nausea, Headache, He...",4.2,Don’t let the name scare you! The only herbici...,False,False,True,True,False,False,True,False,True,False,False,False,True,False,True,True,True,True,False,True,False,False,True,False,False,False,False,True,True,False,False,True


In [17]:
df.shape

(1495, 40)

In [18]:
df.isna().sum()

name                        0
race                        0
flavors                     0
positive                    0
negative                    0
medical                     0
rating                      0
description                 0
positive_aroused            0
positive_creative           0
positive_energetic          0
positive_euphoric           0
positive_focused            0
positive_giggly             0
positive_happy              0
positive_hungry             0
positive_relaxed            0
positive_sleepy             0
positive_talkative          0
positive_tingly             0
positive_uplifted           0
negative_anxious            0
negative_dizzy              0
negative_dry_eyes           0
negative_dry_mouth          0
negative_paranoid           0
symptom_cramps              0
symptom_depression          0
symptom_eye_pressure        0
symptom_fatigue             0
symptom_headaches           0
symptom_inflammation        0
symptom_insomnia            0
symptom_la

# Export dataset as CSV:

In [0]:
df.to_csv('medcabinet_dataset.csv', index=False)